Checking ingletons inequality for the W state on 6 qubits

In [1]:
from QIP import *
n=6
W_state_dict = {}
for i in range(n):
    bitstring = "0"*n
    bitstring = bitstring[:i] + '1' + bitstring[i+1:]
    W_state_dict[bitstring] = 1/n**.5
W_state = QCircuit(6,initial_state=bitstrings_to_vector(W_state_dict))
W_state.subsysem_analysis()
print("W state: "+state_as_string(W_state_dict))
# prints an empty list if ingletons never fails
print("Average failure saturation of ingletons inequality: "+str(W_state.ing["avg_fail_saturation"]))

W state: 0.408|100000> + 0.408|010000> + 0.408|001000> + 0.408|000100> + 0.408|000010> + 0.408|000001>
Average failure saturation of ingletons inequality: []


Checking Ingleton on the state $\ket{\psi}=\frac{1}{4}(\ket{0000}+\ket{1111}+\ket{1010}+\ket{1001})$

In [2]:
state_dict = {}
state_dict['0000']=1/4
state_dict['1111']=1/4
state_dict['1010']=1/4
state_dict['1001']=1/4
ing_test = QCircuit(4,initial_state=bitstrings_to_vector(state_dict))
ing_test.subsysem_analysis()
print("Average failure saturation of ingletons inequality: "+str(ing_test.ing["avg_fail_saturation"]))

Average failure saturation of ingletons inequality: []


Checking Ingleton using $\rho=\frac{1}{2}\ket{\psi}\bra{\psi}+\frac{1}{4}\ket{1010}\bra{1010}+\frac{1}{4}\ket{1001}\bra{1001}$, where $\ket{\psi}=\frac{1}{\sqrt{2}}(\ket{0000}+\bra{1111})$

In [3]:
psi_dict = {}
psi_dict["0000"]=1/2**.5
psi_dict["1111"]=1/2**.5
psi_vec = bitstrings_to_vector(psi_dict)
psi_vec.shape=(2**4,1)
vec1 = bitstrings_to_vector("1010")
vec1.shape=(2**4,1)
vec2 = bitstrings_to_vector("1001")
vec2.shape=(2**4,1)
rho = 0.5*np.outer(psi_vec,psi_vec.conj().T)+0.25*np.outer(vec1,vec1.conj().T)+0.25*np.outer(vec2,vec2.conj().T)

ing_test = QCircuit(4)
ing_test.density_matrix=rho
ing_test.subsysem_analysis()
print("Average failure saturation of ingletons inequality: "+str(ing_test.ing["avg_fail_saturation"]))

Average failure saturation of ingletons inequality: []


In [28]:
entropies_map = {}
subsystems = list(chain.from_iterable(n_partitioning(list(range(4)),2)))
subsystems.sort(key=len)
for subsystem in n_partitioning(list(range(4)),2):
    reduced_dm_A = partial_trace(rho,subsystem[1])
    reduced_dm_B = partial_trace(rho,subsystem[0])
    probs_A = np.linalg.eigvals(reduced_dm_A).tolist()
    probs_B = np.linalg.eigvals(reduced_dm_B).tolist()
    entropy_A = shannon_ent([p_i.real for p_i in probs_A])
    entropy_B = shannon_ent([p_i.real for p_i in probs_B])
    entropies_map[tuple(subsystem[0])] = entropy_A
    # S_A always = S_B if S_AB = 0 (S_AB is pure) so we dont waste time calculating it again
    entropies_map[tuple(subsystem[1])] = entropy_B

entropies_map


{(): -0.0,
 (0, 1, 2, 3): 1.039720770839918,
 (0,): 0.5623351446188083,
 (1, 2, 3): 1.3862943611198906,
 (1,): 0.5623351446188083,
 (0, 2, 3): 1.3862943611198906,
 (2,): 0.6931471805599453,
 (0, 1, 3): 1.3862943611198906,
 (3,): 0.6931471805599453,
 (0, 1, 2): 1.3862943611198906,
 (0, 1): 1.0397207708399179,
 (2, 3): 1.3862943611198906,
 (0, 2): 1.0397207708399179,
 (1, 3): 1.0397207708399179,
 (0, 3): 1.0397207708399179,
 (1, 2): 1.0397207708399179}

In [29]:
mutual_conditional([0],[1],[2],entropies_map)

-1.1102230246251565e-16

In [30]:
mutual_conditional([0],[1],[3],entropies_map)

-1.1102230246251565e-16

In [31]:
mutual([2],[3],entropies_map)

0.0

In [32]:
mutual([0],[1],entropies_map)

0.08494951839769871